<a href="https://colab.research.google.com/github/chloe-florit/chloe-florit.github.io/blob/master/noaa_climate_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import sqlite3
import numpy as np
from plotly import express as px

In [ ]:
temps = pd.read_csv("temps_stacked.csv")
temps['countryID'] = temps['ID'].str[:2]
temps.head()

,ID,Year,Month,Temp,countryID
0,ACW00011604,1961,1,-0.89,AC
1,ACW00011604,1961,2,2.36,AC
2,ACW00011604,1961,3,4.72,AC
3,ACW00011604,1961,4,7.73,AC
4,ACW00011604,1961,5,11.28,AC


In [ ]:
countries = pd.read_csv('countries.csv')
countries.head()

,FIPS 10-4,ISO 3166,Name
0,AF,AF,Afghanistan
1,AX,-,Akrotiri
2,AL,AL,Albania
3,AG,DZ,Algeria
4,AQ,AS,American Samoa


In [ ]:
countries = countries.rename(columns= {"FIPS 10-4": "FIPS_104"})
countries.head()

,FIPS_104,ISO 3166,Name
0,AF,AF,Afghanistan
1,AX,-,Akrotiri
2,AL,AL,Albania
3,AG,DZ,Algeria
4,AQ,AS,American Samoa


In [ ]:
stations = pd.read_csv('station-metadata.csv')
stations.head()

,ID,LATITUDE,LONGITUDE,STNELEV,NAME
0,ACW00011604,57.7667,11.8667,18.0,SAVE
1,AE000041196,25.3330,55.5170,34.0,SHARJAH_INTER_AIRP
2,AEM00041184,25.6170,55.9330,31.0,RAS_AL_KHAIMAH_INTE
3,AEM00041194,25.2550,55.3640,10.4,DUBAI_INTL
4,AEM00041216,24.4300,54.4700,3.0,ABU_DHABI_BATEEN_AIR


In [ ]:
conn = sqlite3.connect("temps.db")
temps.to_sql("temperatures", conn, if_exists="replace", index=False)
countries.to_sql("countries", conn, if_exists="replace", index=False)
stations.to_sql("stations", conn, if_exists="replace", index=False)
# always close your connection
conn.close()

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [ ]:
def query_climate_database(country, year_begin, year_end, month):
    conn = sqlite3.connect("temps.db")

    cmd = \
    """
    SELECT temperatures.Month, temperatures.Year, temperatures.Temp,  stations.NAME, \
    stations.LATITUDE, stations.LONGITUDE, countries.Name
    FROM temperatures
    JOIN stations ON temperatures.ID = stations.ID
    JOIN countries ON countries.FIPS_104 = temperatures.countryID
    WHERE (temperatures.Year>=""" + str(year_begin) + """ ) \
    AND (temperatures.Year<=""" + str(year_end) + """) \
    AND (temperatures.Month= """ + str(month) + """) \
    AND (countries.Name='"""+ country  +"""')

    """
    df = pd.read_sql(cmd, conn)
    return(df)

df1 = query_climate_database(country = "India",
                       year_begin = 1980,
                       year_end = 2020,
                       month = 1)

In [ ]:

df1

,Month,Year,Temp,NAME,LATITUDE,LONGITUDE,Name
0,1,1980,23.48,PBO_ANANTAPUR,14.583,77.633,India
1,1,1981,24.57,PBO_ANANTAPUR,14.583,77.633,India
2,1,1982,24.19,PBO_ANANTAPUR,14.583,77.633,India
3,1,1983,23.51,PBO_ANANTAPUR,14.583,77.633,India
4,1,1984,24.81,PBO_ANANTAPUR,14.583,77.633,India
...,...,...,...,...,...,...,...
3147,1,1983,5.10,DARJEELING,27.050,88.270,India
3148,1,1986,6.90,DARJEELING,27.050,88.270,India
3149,1,1994,8.10,DARJEELING,27.050,88.270,India
3150,1,1995,5.60,DARJEELING,27.050,88.270,India


In [ ]:
from sklearn.linear_model import LinearRegression

def temperature_coefficient_plot(country, year_begin, year_end, month, min_obs, **kwargs):
    monthDict={1:'January', 2:'February', 3:'March', 4:'April', 5:'May', 6:'June', 7:'July', 8:'August', 9:'September', 10:'October', 11:'November', 12:'December'}

    conn = sqlite3.connect("temps.db")

    cmd = \
    """
    SELECT temperatures.Month, temperatures.Year, temperatures.Temp,  stations.NAME, \
    stations.LATITUDE, stations.LONGITUDE, countries.Name
    FROM temperatures
    JOIN stations ON temperatures.ID = stations.ID
    JOIN countries ON countries.FIPS_104 = temperatures.countryID
    WHERE (temperatures.Year>=""" + str(year_begin) + """ ) \
    AND (temperatures.Year<=""" + str(year_end) + """) \
    AND (temperatures.Month= """ + str(month) + """) \
    AND (countries.Name='"""+ country  +"""')

    """
    df = pd.read_sql(cmd, conn)
    df['count']=df.groupby(['NAME',"Month"])['Year'].transform("count")
    df = df[df["count"]>min_obs]

    # your old friend scikit-learn
    def coef(data_group):
      X = data_group[["Year"]]
      y = data_group["Temp"]
      LR = LinearRegression()
      LR.fit(X, y)
      slope = LR.coef_[0]
      return slope

    coefs = df.groupby(['NAME',"Month","LONGITUDE","LATITUDE"]).apply(coef)
    coefs = coefs.reset_index()

    coefs.columns = ['NAME',"Month","LONGITUDE","LATITUDE","Estimated yearly increase (C)"]


    fig = px.scatter_mapbox(coefs, # data for the points you want to plot
                        lat = "LATITUDE", # column name for latitude informataion
                        lon = "LONGITUDE", # column name for longitude information
                        hover_name = "NAME", # what's the bold text that appears when you hover over
                         # how much you want to zoom into the map
                        height = 300, # control aspect ratio

                        # opacity for each data point
                       color="Estimated yearly increase (C)",
                       title = "Estimates of yearly increases in temperature in "+ monthDict.get(month)+\
" for stations in " +country + " from "\
                       +str(year_begin)+"-"+str(year_end),
                       **kwargs) # represent temp using color
    fig.update_layout(margin={"r":0,"t":50,"l":50,"b":0})
    write_html(fig, "example_fig.html")
    return(fig.show())


color_map = px.colors.diverging.RdGy_r
df2 = temperature_coefficient_plot(country = "India",
                       year_begin = 1980,
                       year_end = 2020,
                       month = 1,
                       min_obs = 10,
                       mapbox_style="carto-positron",
                       color_continuous_scale=color_map,
                       zoom = 2)




In [ ]:
temperature_coefficient_plot(country = "France",
                       year_begin = 1980,
                       year_end = 2020,
                       month = 5,
                       min_obs = 10,
                       mapbox_style="carto-positron",
                       color_continuous_scale=color_map,
                       zoom = 3)